## qiita用サンプルNotebook


### 共通処理

In [1]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [2]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np

# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt

# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib

# pandas用ライブラリ
import pandas as pd

# データフレーム表示用関数
from IPython.display import display

In [3]:
# 表示オプション調整

# NumPy表示形式の設定
np.set_printoptions(
    suppress=True, precision=4, floatmode='fixed'
)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 方眼表示ON
plt.rcParams['axes.grid'] = True


# データフレームでの表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns", None)

###  テンプレートファイルのコピー

In [12]:
url1 = 'https://github.com/makaishi2/sample-data/raw/master/data/bank-marketing-template.xlsx'
!wget $url1
!ls -l

--2023-06-03 07:15:15--  https://github.com/makaishi2/sample-data/raw/master/data/bank-marketing-template.xlsx
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/makaishi2/sample-data/master/data/bank-marketing-template.xlsx [following]
--2023-06-03 07:15:15--  https://raw.githubusercontent.com/makaishi2/sample-data/master/data/bank-marketing-template.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9678 (9.5K) [application/octet-stream]
Saving to: ‘bank-marketing-template.xlsx’

bank-marketing-temp 100%[===================>]   9.45K  --.-KB/s    in 0s      

2023-06-03 07:15:15 (79.9 MB/s)

### データ読み込み

In [13]:
url2 = 'https://github.com/makaishi2/sample-data/raw/master/data/bank-marketing-v2.csv'
df = pd.read_csv(url2)

In [14]:
df.isnull().sum()

年齢          0
職業        288
婚姻          0
学歴       1857
平均残高        0
住宅ローン       0
個人ローン       0
dtype: int64

In [15]:
rows = df.shape[0]
cols = df.shape[1]
max_row = rows + 1
print(f'cols: {cols} rows: {rows}')
display(df.head())

cols: 7 rows: 45211


,年齢,職業,婚姻,学歴,平均残高,住宅ローン,個人ローン
0,58,management,married,tertiary,2143,True,False
1,44,technician,single,secondary,29,True,False
2,33,entrepreneur,married,secondary,2,True,True
3,47,blue-collar,married,NaN,1506,True,False
4,33,NaN,single,NaN,1,False,False


### Excelファイルオープン

In [16]:
!cp bank-marketing-template.xlsx bank-marketing-record.xlsx

In [17]:
fn = 'bank-marketing-record.xlsx'

# コピー後のExcelオープン
from openpyxl import load_workbook
wb = load_workbook(fn)

# シート選択(1シートしかないのでこれでOK)
ws = wb.active

### セル値のコピー

In [19]:
# セル値のコピー
for row in range(rows):
    for col in range(cols):
        ws.cell(row=row+2, column=col+1).value = df.iloc[row, col]

### 罫線設定

In [20]:
from openpyxl.styles.borders import Border, Side
# 罫線設定用
side = Side(style='thin', color='000000')
border_all = Border(top=side, bottom=side, left=side, right=side)

for row in range(rows+1):
    for col in range(cols+3):
        ws.cell(row=row+1, column=col+1).border = border_all

###  DataValidation設定

In [21]:
from openpyxl.worksheet.datavalidation import DataValidation

dv1 = DataValidation(
    type="list",
    formula1='"1. 見込みあり,2. 優先度低,3. 見込みなし"',
    showErrorMessage=True,
    errorStyle="stop",
)

dv2 = DataValidation(
    type="list",
    formula1='"1. 成功,2. 失敗"',
    showErrorMessage=True,
    errorStyle="stop",
)

# 見込み
dv1.add(f'H2:H{max_row}')
ws.add_data_validation(dv1)

# 成功
dv2.add(f'I2:I{max_row}')
ws.add_data_validation(dv2)

### セルロック設定

In [22]:
from openpyxl.styles import Protection

# ワークシート全体にロックをかける
ws.protection.enable()
# フィルターは認める
ws.protection.autoFilter = False

# 特定セルに対しロック解除
unlock_cells = ws[f'H2:J{max_row}']
for row in unlock_cells:
    for cell in row:
        cell.protection = Protection(locked=False)

unlock_cells = ws[f'A1:J1']
for row in unlock_cells:
    for cell in row:
        cell.protection = Protection(locked=False)

### 計算式設定

In [27]:
# 全体件数
ws['M1'].value = f'=counta(A2:A{max_row})'

#　成功件数
ws['O1'].value = f'=countif(I2:I{max_row},"1. 成功")'

#　失敗件数
ws['Q1'].value = f'=countif(I2:I{max_row},"2. 失敗")'

### 保存

In [28]:
# ファイル保存
wb.save(fn)